# Opening the .xml's

In [1]:
import math
import os
import requests

from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import numpy as np
import pandas as pd

In [2]:
# File names in list that I've downloaded into .xml's
abc_xml = ['abc001_10.xml', 'abc001_11.xml','abc001_12.xml','abc001_13.xml','abc001_14.xml', 'abc002_1.xml',]
dna_xml = ['dna001_1.xml', 'dna002_4.xml', 'dna004_2.xml']

# Files names to write the .csv's
abc_csv = ['abc001_10.csv', 'abc001_11.csv','abc001_12.csv','abc001_13.csv','abc001_14.csv', 'abc002_1.csv',]
dna_csv = ['dna001_1.csv', 'dna002_4.csv', 'dna004_2.csv']

In [ ]:
# Open and read the .xml from the BLAST
result_handle = open(abc_xml[1])
blast_record = NCBIXML.read(result_handle)

# Initiate arrays
hit_names = []
hit_e_values = []
hit_scores = []
hit_norm_scores = []

# Add alginment data to hits_array
E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            alignment_data = ['string0', 'string1', 'string2']
            alignment_data[0] = alignment.title
            alignment_data[1] = hsp.expect
            alignment_data[2] = hsp.score
            #make expect and score floats from strings
            alignment_data[1] = float(alignment_data[1])
            alignment_data[2] = float(alignment_data[2])
            #append the alignment data to the database hits array
            pdb_hits_array.append(alignment_data)

# Convert Python array to Pandas DataFrame
column_names = ['hit_title', 'e_value', 'score']
pdb_hits_df = pd.DataFrame(pdb_hits_array, columns=column_names)

# Calculate the normalized score
norm_score = [100]
for i in range(1,len(pdb_hits_array)):
    norm_score_i = pdb_hits_df['score'][i] / pdb_hits_df['score'][0] *100
    norm_score.append(norm_score_i)

# Add Pandas Series from this list of normalized scores to the hits DataFrame
pdb_hits_df['norm_score'] = pd.Series(norm_score)

# Convert Pandas DataFrame to .csv
pdb_hits_df.to_csv('dna002_4.csv')

# Don't look past here

## Open the .xml's in the same directory as this notebook

In [3]:
# Open and read the .xml from the BLAST
result_handle = open('dna002_4.xml')
blast_record = NCBIXML.read(result_handle)

In [4]:
# Use Numpy array to store data
# Initiate array
pdb_hits_array = []

# Leave out the normalized score!!  Hard to iterate through when not iterating integers

# Add alginment data to hits_array
E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000004
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            alignment_data = ['string0', 'string1', 'string2']
            alignment_data[0] = alignment.title
            alignment_data[1] = hsp.expect
            alignment_data[2] = hsp.score
            #make expect and score floats from strings
            alignment_data[1] = float(alignment_data[1])
            alignment_data[2] = float(alignment_data[2])
            #append the alignment data to the database hits array
            pdb_hits_array.append(alignment_data)

# Convert Python array to Pandas DataFrame
column_names = ['hit_title', 'e_value', 'score']
pdb_hits_df = pd.DataFrame(pdb_hits_array, columns=column_names)

# Calculate the normalized score
norm_score = [100]
for i in range(1,len(pdb_hits_array)):
    norm_score_i = pdb_hits_df['score'][i] / pdb_hits_df['score'][0] *100
    norm_score.append(norm_score_i)

# Add Pandas Series from this list of normalized scores to the hits DataFrame
pdb_hits_df['norm_score'] = pd.Series(norm_score)

# Convert Pandas DataFrame to .csv
pdb_hits_df.to_csv('dna002_4.csv')

In [21]:
pdb_hits_array

[]

## Printing hits_results using tutorial code

In [12]:
result_handle = open('abc001_13.xml')

blast_record = NCBIXML.read(result_handle)

# Getting info out of code
E_VALUE_THRESH = 0.0000000000000000000000000000000000000000000000001
count = 0
for alignment in blast_record.alignments:
    count += 1
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence:', alignment.title)
            print('e value:', hsp.expect)
            print('score:', hsp.score)
            #print('identities:', hsp.identities)
print('There are', count, 'sequences in the BLAST output')

There are 4 sequences in the BLAST output
